In [2]:
import os
import sys
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import ARMAConv
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, log_loss, roc_auc_score
)
import warnings
warnings.filterwarnings("ignore")

SEED = 42
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.set_num_threads(4)

fa_cn = "/home/snu/Downloads/Histogram_CN_FA_20bin_updated.npy"
fa_ad = "/home/snu/Downloads/Histogram_MCI_FA_20bin_updated.npy"

# ------------------------
# Load features & labels
# ------------------------
X_cn = np.load(fa_cn, allow_pickle=True)
X_ad = np.load(fa_ad, allow_pickle=True)

# CN → label 0, AD → label 1
X = np.vstack([X_cn, X_ad]).astype(np.float32)
y = np.hstack([
    np.zeros(X_cn.shape[0], dtype=np.int64),
    np.ones(X_ad.shape[0], dtype=np.int64)
])

# Shuffle dataset
perm = np.random.RandomState(SEED).permutation(len(X))
X = X[perm]
y = y[perm]

num_nodes, num_feats = X.shape
print(f"Dataset: CN vs AD — nodes={num_nodes}, feats={num_feats}")

# ------------------------
# Adjacency building
# ------------------------
def create_adj(F, alpha=1.0):
    norms = np.linalg.norm(F, axis=1, keepdims=True)
    norms[norms == 0] = 1.0
    F_norm = F / norms
    W = np.dot(F_norm, F_norm.T)
    W = (W >= alpha).astype(np.float32)
    return W

W0 = create_adj(X, alpha=0.8)
print(f"W0: {W0.shape}")

def load_graph_torch(adj, node_feats):
    node_feats_t = torch.from_numpy(node_feats).float()
    edge_idx = np.array(np.nonzero(adj))
    edge_index = torch.from_numpy(edge_idx).long()
    return node_feats_t, edge_index

node_feats_all, edge_index_all = load_graph_torch(W0, X)
print(f"Number of edges: {edge_index_all.size(1)}")

# ------------------------
# Model components
# ------------------------
class ARMAEncoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, num_stacks=2, num_layers=1, activ="RELU"):
        super(ARMAEncoder, self).__init__()
        activations = {
            "SELU": F.selu,
            "SiLU": F.silu,
            "GELU": F.gelu,
            "ELU": F.elu,
            "RELU": F.relu
        }
        self.act = activations.get(activ, F.elu)
        self.arma = ARMAConv(
            input_dim, hidden_dim,
            num_stacks=num_stacks,
            num_layers=num_layers,
            shared_weights=True,
            dropout=0.3
        )
        self.batchnorm = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(0.3)
        self.mlp = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.arma(x, edge_index)
        x = self.act(x)
        x = self.dropout(x)
        x = self.batchnorm(x)
        logits = self.mlp(x)
        return logits

class AvgReadout(nn.Module):
    def forward(self, seq, msk=None):
        if msk is None:
            return torch.mean(seq, 0)
        else:
            msk = torch.unsqueeze(msk, -1)
            return torch.sum(seq * msk, 0) / torch.sum(msk)

class Discriminator(nn.Module):
    def __init__(self, n_h):
        super().__init__()
        self.f_k = nn.Bilinear(n_h, n_h, 1)
        nn.init.xavier_uniform_(self.f_k.weight.data)
        if self.f_k.bias is not None:
            self.f_k.bias.data.fill_(0.0)

    def forward(self, c, h_pl, h_mi):
        c_x = torch.unsqueeze(c, 0).expand_as(h_pl)
        sc_1 = torch.squeeze(self.f_k(h_pl, c_x), 1)
        sc_2 = torch.squeeze(self.f_k(h_mi, c_x), 1)
        return torch.cat((sc_1, sc_2), 0)

class DGI(nn.Module):
    def __init__(self, n_in, n_h, num_stacks=2, num_layers=1):
        super().__init__()
        self.gcn1 = ARMAEncoder(n_in, n_h, num_stacks=num_stacks, num_layers=num_layers)
        self.read = AvgReadout()
        self.sigm = nn.Sigmoid()
        self.disc = Discriminator(n_h)

    def forward(self, seq1, seq2, edge_index):
        data1 = Data(x=seq1, edge_index=edge_index)
        data2 = Data(x=seq2, edge_index=edge_index)
        h_1 = self.gcn1(data1)
        c = self.read(h_1)
        c = self.sigm(c)
        h_2 = self.gcn1(data2)
        logits = self.disc(c, h_1, h_2)
        return logits, h_1

class DGI_with_classifier(DGI):
    def __init__(self, n_in, n_h, n_classes=2, cut=0, device='cpu', num_stacks=2, num_layers=1):
        super().__init__(n_in, n_h, num_stacks=num_stacks, num_layers=num_layers)
        self.classifier = nn.Linear(n_h, n_classes)
        self.cut = cut
        self.device = device
        self.n_clusters = n_classes

    def get_embeddings(self, node_feats, edge_index):
        _, embeddings = self.forward(node_feats, node_feats, edge_index)
        return embeddings

    def cut_loss(self, A, S):
        C = F.softmax(S, dim=1)
        A_pool = torch.matmul(torch.matmul(A, C).t(), C)
        num = torch.trace(A_pool)
        D = torch.diag(torch.sum(A, dim=-1))
        D_pooled = torch.matmul(torch.matmul(D, C).t(), C)
        den = torch.trace(D_pooled) + 1e-9
        mincut_loss = -(num / den)
        St_S = torch.matmul(C.t(), C)
        I_S = torch.eye(C.shape[1], device=A.device)
        ortho_loss = torch.norm(St_S / (torch.norm(St_S) + 1e-9) - I_S / (torch.norm(I_S) + 1e-9))
        return mincut_loss + ortho_loss

    def modularity_loss(self, A, S):
        C = F.softmax(S, dim=1)
        d = torch.sum(A, dim=1)
        m = torch.sum(A) + 1e-9
        B = A - torch.outer(d, d) / (2 * m)
        I_S = torch.eye(C.shape[1], device=A.device)
        k = torch.norm(I_S)
        n = S.shape[0]
        modularity_term = (-1 / (2 * m)) * torch.trace(torch.mm(torch.mm(C.t(), B), C))
        collapse_reg_term = (torch.sqrt(k) / n) * torch.norm(torch.sum(C, dim=0), p='fro') - 1
        return modularity_term + collapse_reg_term

    def Reg_loss(self, A, embeddings):
        logits = self.classifier(embeddings)
        if self.cut == 1:
            return self.cut_loss(A, logits)
        else:
            return self.modularity_loss(A, logits)

# ------------------------
# Prepare cross-validation
# ------------------------
sss = StratifiedShuffleSplit(n_splits=20, test_size=0.5, random_state=SEED)
accuracies, precisions, recalls, f1_scores, losses, all_auc = [], [], [], [], [], []

def get_device():
    try:
        if torch.cuda.is_available():
            dev = torch.device("cuda")
            torch.tensor([1.0], device=dev) + 1.0
            return dev
    except Exception as e:
        print("CUDA not usable, falling back to CPU:", e)
        try:
            torch.cuda.empty_cache()
        except:
            pass
    return torch.device("cpu")

device = get_device()
print("Using device:", device)

A_tensor = torch.from_numpy(W0).float().to(device)
hidden_dim = 512
cut = 1
num_epochs = 5000
lr = 1e-4
weight_decay = 1e-4
reg_weight = 0.1
num_stacks = 2
num_layers = 1

node_feats = node_feats_all.to(device)
edge_index = edge_index_all.to(device)

N = node_feats.size(0)
lbl = torch.cat([torch.ones(N, device=device), torch.zeros(N, device=device)]).float()

for fold, (train_idx, test_idx) in enumerate(sss.split(X, y), start=1):
    print(f"\n=== Fold {fold} (CN vs AD) ===")

    cn_idx = np.where(y == 0)[0]
    ad_idx = np.where(y == 1)[0]

    sss_class = StratifiedShuffleSplit(n_splits=20, test_size=0.1, random_state=fold)
    cn_train_idx, cn_test_idx = next(sss_class.split(X[cn_idx], y[cn_idx]))
    ad_train_idx, ad_test_idx = next(sss_class.split(X[ad_idx], y[ad_idx]))

    cn_train = cn_idx[cn_train_idx]
    ad_train = ad_idx[ad_train_idx]
    cn_test = cn_idx[cn_test_idx]
    ad_test = ad_idx[ad_test_idx]

    balanced_train_idx = np.concatenate([cn_train, ad_train])
    test_idx_final = np.concatenate([cn_test, ad_test])
    np.random.shuffle(balanced_train_idx)
    np.random.shuffle(test_idx_final)

    print(f"Train CN: {len(cn_train)}, Train AD: {len(ad_train)}")
    print(f"Test CN: {len(cn_test)}, Test AD: {len(ad_test)}")

    y_tensor = torch.from_numpy(y).long().to(device)
    train_idx_t = torch.from_numpy(balanced_train_idx).long().to(device)
    test_idx_t = torch.from_numpy(test_idx_final).long().to(device)

    model = DGI_with_classifier(num_feats, hidden_dim, n_classes=2, cut=cut,
                                device=device, num_stacks=num_stacks, num_layers=num_layers).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    bce_loss = nn.BCEWithLogitsLoss()
    ce_loss = nn.CrossEntropyLoss()

    for epoch in range(1, num_epochs + 1):
        model.train()
        optimizer.zero_grad()
        perm = torch.randperm(N, device=device)
        corrupt = node_feats[perm]
        logits_dgi, embeddings = model(node_feats, corrupt, edge_index)
        dgi_loss = bce_loss(logits_dgi.squeeze(), lbl)
        logits_cls = model.classifier(embeddings)
        train_logits = logits_cls[train_idx_t]
        train_labels = y_tensor[train_idx_t]
        supervised_loss = ce_loss(train_logits, train_labels)
        reg_loss_val = model.Reg_loss(A_tensor, embeddings)
        total_loss = supervised_loss + dgi_loss + reg_weight * reg_loss_val
        total_loss.backward()
        optimizer.step()

        if epoch % 500 == 0 or epoch == 1:
            model.eval()
            with torch.no_grad():
                logits_eval = model.classifier(model.get_embeddings(node_feats, edge_index))
                preds_train = torch.argmax(logits_eval[train_idx_t], dim=1).cpu().numpy()
                acc = accuracy_score(train_labels.cpu().numpy(), preds_train)
            print(f"Epoch {epoch}: Sup={supervised_loss.item():.4f} | DGI={dgi_loss.item():.4f} | "
                  f"Reg={reg_loss_val.item():.4f} | Total={total_loss.item():.4f} | TrainAcc={acc:.4f}")

    model.eval()
    with torch.no_grad():
        emb_final = model.get_embeddings(node_feats, edge_index)
        logits_final = model.classifier(emb_final)
        probs = F.softmax(logits_final, dim=1).cpu().numpy()
        y_pred = np.argmax(probs, axis=1)

    y_test = y[test_idx_final]
    y_pred_test = y_pred[test_idx_final]
    y_proba_test = probs[test_idx_final, 1]

    acc = accuracy_score(y_test, y_pred_test)
    prec = precision_score(y_test, y_pred_test, zero_division=0)
    rec = recall_score(y_test, y_pred_test, zero_division=0)
    f1 = f1_score(y_test, y_pred_test, zero_division=0)
    loss_val = log_loss(y_test, y_proba_test)
    auc_score = roc_auc_score(y_test, y_proba_test)

    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1_scores.append(f1)
    losses.append(loss_val)
    all_auc.append(auc_score)

    print(f"Fold {fold} → Acc={acc:.4f} Prec={prec:.4f} Rec={rec:.4f} F1={f1:.4f} AUC={auc_score:.4f}")

print("\n=== Average Results (CN vs AD) ===")
print(f"Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
print(f"Precision: {np.mean(precisions):.4f} ± {np.std(precisions):.4f}")
print(f"Recall: {np.mean(recalls):.4f} ± {np.std(recalls):.4f}")
print(f"F1: {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
print(f"LogLoss: {np.mean(losses):.4f} ± {np.std(losses):.4f}")
print(f"AUC: {np.mean(all_auc):.4f} ± {np.std(all_auc):.4f}")


Dataset: CN vs AD — nodes=300, feats=180
W0: (300, 300)
Number of edges: 79964
Using device: cuda

=== Fold 1 (CN vs AD) ===
Train CN: 119, Train AD: 150
Test CN: 14, Test AD: 17
Epoch 1: Sup=0.7011 | DGI=0.7096 | Reg=-0.2363 | Total=1.3870 | TrainAcc=0.4424
Epoch 500: Sup=0.3747 | DGI=0.6937 | Reg=-0.3689 | Total=1.0315 | TrainAcc=0.8699
Epoch 1000: Sup=0.3042 | DGI=0.6946 | Reg=-0.3841 | Total=0.9604 | TrainAcc=0.9368
Epoch 1500: Sup=0.2452 | DGI=0.6934 | Reg=-0.3908 | Total=0.8995 | TrainAcc=0.9517
Epoch 2000: Sup=0.2656 | DGI=0.7044 | Reg=-0.4087 | Total=0.9291 | TrainAcc=0.9628
Epoch 2500: Sup=0.1956 | DGI=0.6932 | Reg=-0.4273 | Total=0.8460 | TrainAcc=0.9888
Epoch 3000: Sup=0.2444 | DGI=0.6936 | Reg=-0.4311 | Total=0.8949 | TrainAcc=0.9963
Epoch 3500: Sup=0.1360 | DGI=0.6946 | Reg=-0.4243 | Total=0.7882 | TrainAcc=0.9926
Epoch 4000: Sup=0.2010 | DGI=0.6932 | Reg=-0.4169 | Total=0.8525 | TrainAcc=0.9888
Epoch 4500: Sup=0.2105 | DGI=0.6932 | Reg=-0.4161 | Total=0.8620 | TrainAcc=1.